In [1]:
from keras.models import Model, load_model, model_from_json
from keras.layers import Input, Cropping2D, Dropout
from keras.layers.core import Lambda
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint, History
from keras import backend as K
from keras.optimizers import Nadam
from keras.utils import multi_gpu_model

import tensorflow as tf

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
list_gpus = K.tensorflow_backend._get_available_gpus()
print(list_gpus)


['/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1']


In [3]:
import threading

In [4]:
import numpy as np
#from sklearn.model_selection import StratifiedShuffleSplit,ShuffleSplit
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

In [5]:
import sys
sys.path.insert(0, '/nfs/src')
from features import geopatches

In [6]:
import datetime
import os

In [7]:
import pandas as pd
import h5py

In [8]:
# Setup data seed
np.random.seed(3254)

In [9]:
# https://github.com/ternaus/kaggle_dstl_submission/

smooth = 1e-12

def jaccard_coef(y_true, y_pred):
    intersection = K.sum(y_true * y_pred, axis=[0, -1, -2])
    sum_ = K.sum(y_true + y_pred, axis=[0, -1, -2])

    jac = (intersection + smooth) / (sum_ - intersection + smooth)

    return K.mean(jac)


def jaccard_coef_int(y_true, y_pred):
    y_pred_pos = K.round(K.clip(y_pred, 0, 1))

    intersection = K.sum(y_true * y_pred_pos, axis=[0, -1, -2])
    sum_ = K.sum(y_true + y_pred_pos, axis=[0, -1, -2])

    jac = (intersection + smooth) / (sum_ - intersection + smooth)

    return K.mean(jac)


def jaccard_coef_loss(y_true, y_pred):
    return -K.log(jaccard_coef(y_true, y_pred)) + K.binary_crossentropy(y_pred, y_true)
#    return -K.log(jaccard_coef(y_true, y_pred))

def save_model(model, cross, results_dir):
    json_string = model.to_json()
    if not os.path.isdir(os.path.join(results_dir,'cache')):
        os.makedirs(os.path.join(results_dir,'cache'))
    json_name = 'architecture_' + cross + '.json'
    weight_name = 'model_weights_' + cross + '.h5'
    open(os.path.join(results_dir,'cache', json_name), 'w').write(json_string)
    model.save_weights(os.path.join(results_dir,'cache', weight_name), overwrite=True)

def save_history(history, cross, results_dir):
    if not os.path.isdir(os.path.join(results_dir,'history')):
        os.makedirs(os.path.join(results_dir,'history'))
    filename = os.path.join(results_dir,'history') + '/history_' + cross + '.csv'
    pd.DataFrame(history.history).to_csv(filename, index=False)

def read_model(cross,results_dir):
    json_name = 'architecture_' + cross + '.json'
    weight_name = 'model_weights_' + cross + '.h5'
    model = model_from_json(open(os.path.join(results_dir,'cache', json_name)).read())
    model.load_weights(os.path.join(results_dir,'cache', weight_name))
    return model

In [10]:
def model_unet(img_cols, img_rows, nb_channels, img_crop):
    
    input_shape = (img_cols, img_rows, nb_channels)
    inputs = Input(input_shape)

    c1 = Conv2D(8, (3, 3), activation='elu', padding='same') (inputs)
    c1 = BatchNormalization(axis=1)(c1)
    c1 = Dropout(0.1) (c1)
    c1 = Conv2D(8, (3, 3), activation='elu', padding='same') (c1)
    c1 = BatchNormalization(axis=1)(c1)
    p1 = MaxPooling2D((2, 2)) (c1)

    c2 = Conv2D(16, (3, 3), activation='elu', padding='same') (p1)
    c2 = BatchNormalization(axis=1)(c2)
    c2 = Dropout(0.1) (c2)
    c2 = Conv2D(16, (3, 3), activation='elu', padding='same') (c2)
    c2 = BatchNormalization(axis=1)(c2)
    p2 = MaxPooling2D((2, 2)) (c2)

    c3 = Conv2D(32, (3, 3), activation='elu', padding='same') (p2)
    c3 = BatchNormalization(axis=1)(c3)
    c3 = Dropout(0.1) (c3)
    c3 = Conv2D(32, (3, 3), activation='elu', padding='same') (c3)
    c3 = BatchNormalization(axis=1)(c3)
    p3 = MaxPooling2D((2, 2)) (c3)

    c4 = Conv2D(64, (3, 3), activation='elu', padding='same') (p3)
    c4 = BatchNormalization(axis=1)(c4)
    c4 = Dropout(0.1) (c4)
    c4 = Conv2D(64, (3, 3), activation='elu', padding='same') (c4)
    c4 = BatchNormalization(axis=1)(c4)
    p4 = MaxPooling2D(pool_size=(2, 2)) (c4)

    c5 = Conv2D(128, (3, 3), activation='elu', padding='same') (p4)
    c5 = BatchNormalization(axis=1)(c5)
    c5 = Dropout(0.1) (c5)
    c5 = Conv2D(128, (3, 3), activation='elu', padding='same') (c5)
    c5 = BatchNormalization(axis=1)(c5)

    u6 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same') (c5)
    u6 = concatenate([u6, c4])
    c6 = Conv2D(64, (3, 3), activation='elu', padding='same') (u6)
    c6 = BatchNormalization(axis=1)(c6)
    c6 = Dropout(0.1) (c6)
    c6 = Conv2D(64, (3, 3), activation='elu', padding='same') (c6)
    c6 = BatchNormalization(axis=1)(c6)

    u7 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same') (c6)
    u7 = concatenate([u7, c3])
    c7 = Conv2D(32, (3, 3), activation='elu', padding='same') (u7)
    c7 = BatchNormalization(axis=1)(c7)
    c7 = Dropout(0.1) (c7)
    c7 = Conv2D(32, (3, 3), activation='elu', padding='same') (c7)
    c7 = BatchNormalization(axis=1)(c7)

    u8 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same') (c7)
    u8 = concatenate([u8, c2])
    c8 = Conv2D(16, (3, 3), activation='elu', padding='same') (u8)
    c8 = BatchNormalization(axis=1)(c8)
    c8 = Dropout(0.1) (c8)
    c8 = Conv2D(16, (3, 3), activation='elu', padding='same') (c8)
    c8 = BatchNormalization(axis=1)(c8)

    u9 = Conv2DTranspose(8, (2, 2), strides=(2, 2), padding='same') (c8)
    u9 = concatenate([u9, c1], axis=3)
    c9 = Conv2D(8, (3, 3), activation='relu', padding='same') (u9)
    c9 = BatchNormalization(axis=1)(c9)
    c9 = Dropout(0.1) (c9)
    c9 = Conv2D(8, (3, 3), activation='relu', padding='same') (c9)
    c9 = BatchNormalization(axis=1)(c9)

    #outputs = Conv2D(25, (1, 1), activation='sigmoid') (c9)
    c10 = Conv2D(1, (1, 1), activation='sigmoid') (c9)

    outputs = Cropping2D(cropping=((img_crop, img_crop), (img_crop, img_crop))) (c10)
    model = Model(inputs=[inputs], outputs=[outputs])
    
    return model

In [11]:
#https://keunwoochoi.wordpress.com/2017/08/24/tip-fit_generator-in-keras-how-to-parallelise-correctly/

class threadsafe_iter:
    """Takes an iterator/generator and makes it thread-safe by
    serializing call to the `next` method of given iterator/generator.
    """
    def __init__(self, it):
        self.it = it
        self.lock = threading.Lock()

    def __iter__(self):
        return self

    def __next__(self):
        return self.next()
    
    def next(self):
        with self.lock:
            #return self.it.next()
            return next(self.it)


def threadsafe_generator(f):
    """A decorator that takes a generator function and makes it thread-safe.
    """
    def g(*a, **kw):
        return threadsafe_iter(f(*a, **kw))
    return g

@threadsafe_generator
def generate_minibatch(X_train, Y_train, batch_size):
    while 1:
        # Generate one minibatch
        X_batch = np.empty((batch_size, X_train.shape[1], X_train.shape[2], X_train.shape[3]))
        Y_batch = np.empty((batch_size, Y_train.shape[1], Y_train.shape[2], Y_train.shape[3]))
        for i in range(batch_size):
            # Select a random image
            idx = np.random.choice(X_train.shape[0])
            image = X_train[idx]
            label = Y_train[idx]

            if np.random.choice(2) == 0:
                # Flip vertically
                image = np.flipud(image)
                label = np.flipud(label)
            if np.random.choice(2) == 0:
                # Flip horizontally
                image = np.fliplr(image)
                label = np.fliplr(label)

            num_rot = np.random.choice(4)
            image = np.rot90(image, num_rot)
            label = np.rot90(label, num_rot)

            X_batch[i] = image
            Y_batch[i] = label

        yield (X_batch, Y_batch)

In [12]:
def encode_binary_data(dict_replace):
    # Encode classes
    print(dict_replace)
    u,inv = np.unique(Y_raw,return_inverse = True)
    Y = np.array([dict_replace.get(x, 0.0) for x in u])[inv].reshape(Y_raw.shape)
    Y = Y[..., np.newaxis]
    X = X_raw
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.20, random_state=3254)
    X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.10, random_state=3254)
    
    for data,name in zip([Y_train,Y_val,Y_test],['Y_train','Y_val','Y_test']):
        num_img = 0.0
        num_pixels = 0.0
        for x in range(data.shape[0]):
            unique, counts = np.unique(data[x], return_counts=True)
            if len(unique) > 1:
                #print(x, dict(zip(unique, counts)))
                num_img += 1
                num_pixels += dict(zip(unique, counts))[1]
        print ('composition with label != 0')
        print (name, num_img/data.shape[0], num_pixels/(data.shape[0] * data.shape[1] * data.shape[2]))
    return X_train, X_val, X_test, Y_train, Y_val, Y_test

In [13]:
if __name__ == '__main__':
    
    results_dir = '/nfs/results/unet/dropouttest/80_80_16/results_prelim_unet_pl_veg'
    
    dict_img = {'width': 80, 'height': 80, 'stride': 80, 'padding': 16}
    dict_label = {'width': 80, 'height': 80, 'stride': 80}
    X_raw, Y_raw = geopatches.create_data(['/nfs/conditioned/planetlabs/1154314_2014-07-23_RE2_3A_Analytic_clipped.tif'],dict_img,\
                                        ['/nfs/conditioned/landfire/US_140EVT_020818_UTM11_30_clipped.tif'],dict_label,\
                                         repeatlabel=6)
    # Planetlabs needs to be divided by max uint16 = 65535 to be between 0 and 1.0
    X_raw = X_raw / 65535

    #remove classes that are too small
    list_remove = []
    for x in np.asarray(np.unique(Y_raw, return_counts=True)).T:
        if x[1] < 400*36:
            list_remove.append(x[0])
    print('unique labels found:')
    list_unique = np.unique(Y_raw)
    print(list_unique)
    print('labels that do not meet threshold')
    print(list_remove)
    list_final = [x for x in list_unique if x not in list_remove]
    print('final labels')
    print(list_final)
    print('\n')

    # Variables
    batch_size = 128
    nb_epoch = 50
    steps_per_epoch = 100
    window_size = X_raw.shape[1]
    #nb_classes = 1
    nb_channels = X_raw.shape[3]
    cropping_len = 16
    
    # Build models
    for label in list_final:
        
        # Encode
        print('label:',label)
        X_train, X_val, X_test, Y_train, Y_val, Y_test = encode_binary_data({label: 1.0})
        print('X_train.shape','X_val.shape','X_test.shape','Y_train.shape','Y_val.shape','Y_test.shape',':')
        print(X_train.shape,X_val.shape,X_test.shape,Y_train.shape,Y_val.shape,Y_test.shape)
        
        # Model
        # https://www.pyimagesearch.com/2017/10/30/how-to-multi-gpu-training-with-keras-python-and-deep-learning/
        if len(list_gpus) <= 1:
            model = model_unet(window_size,window_size,nb_channels,cropping_len)
            model_gpu = model
        else:
            with tf.device("/cpu:0"):
                model = model_unet(window_size,window_size,nb_channels,cropping_len)
            model_gpu = multi_gpu_model(model, gpus=len(list_gpus))
        
        model_gpu.compile(optimizer=Nadam(lr=1e-3), loss=jaccard_coef_loss,\
                      metrics=['binary_crossentropy', jaccard_coef_int, "binary_accuracy"])
        
        history = History()
        ts_gen = generate_minibatch(X_train, Y_train, batch_size)
        #K.set_session(K.tf.Session(config=K.tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)))
        model_gpu.fit_generator(ts_gen,\
                    nb_epoch=nb_epoch,\
                    verbose=1,\
                    steps_per_epoch=steps_per_epoch,\
                    validation_data=(X_val, Y_val),\
                   callbacks=[history])

        # Calculate metrics off test
        Y_predict = model_gpu.predict(X_test)
        print(accuracy_score(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
        print(classification_report(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
        
        
        # Save
        suffix = str(label)
        save_model(model, "{batch}_{step}_{epoch}_{suffix}".format(batch=batch_size, step=steps_per_epoch, epoch=nb_epoch, suffix=suffix),results_dir)
        save_history(history, "{batch}_{step}_{epoch}_{suffix}".format(batch=batch_size, step=steps_per_epoch, epoch=nb_epoch, suffix=suffix),results_dir)
        
        # Calculate metrics off saved model
#         Y_predict = read_model("{batch}_{step}_{epoch}_{suffix}".format(batch=batch_size, step=steps_per_epoch, epoch=nb_epoch, suffix=suffix),results_dir).predict(X_test)
#         print(accuracy_score(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
#         print(classification_report(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
        
        print('\n')
        

unique labels found:
[3014 3092 3096 3097 3099 3110 3118 3129 3130 3152 3155 3181 3182 3183
 3184 3221 3292 3294 3296 3297 3298 3299 3900 3901 3902 3903 3904 3910
 3911 3912 3913 3914 3923 3924 3926 3927 3928 3929 3964 3984]
labels that do not meet threshold
[3014, 3096, 3155, 3181, 3182, 3183, 3221, 3901, 3911, 3923, 3924, 3926, 3927, 3928, 3964, 3984]
final labels
[3092, 3097, 3099, 3110, 3118, 3129, 3130, 3152, 3184, 3292, 3294, 3296, 3297, 3298, 3299, 3900, 3902, 3903, 3904, 3910, 3912, 3913, 3914, 3929]


label: 3092
{3092: 1.0}
composition with label != 0
Y_train 0.5410628019323671 0.1474718196457327
composition with label != 0
Y_val 0.6 0.19891964285714286
composition with label != 0
Y_test 0.5202312138728323 0.12354407514450867
X_train.shape X_val.shape X_test.shape Y_train.shape Y_val.shape Y_test.shape :
(621, 112, 112, 5) (70, 112, 112, 5) (173, 112, 112, 5) (621, 80, 80, 1) (70, 80, 80, 1) (173, 80, 80, 1)


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:59: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:59: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<__main__...., callbacks=[<keras.ca..., validation_data=(array([[[..., epochs=50, steps_per_epoch=100, verbose=1)`


Epoch 1/50
100/100 [==============================] - 21s 214ms/step - loss: 5.1301 - binary_crossentropy: 0.4626 - jaccard_coef_int: 0.1359 - binary_accuracy: 0.8364 - val_loss: 6.0722 - val_binary_crossentropy: 0.8580 - val_jaccard_coef_int: 0.0538 - val_binary_accuracy: 0.7889
Epoch 2/50
100/100 [==============================] - 13s 128ms/step - loss: 4.0896 - binary_crossentropy: 0.6051 - jaccard_coef_int: 0.2702 - binary_accuracy: 0.8372 - val_loss: 7.6900 - val_binary_crossentropy: 1.0175 - val_jaccard_coef_int: 0.0049 - val_binary_accuracy: 0.8010
Epoch 3/50
100/100 [==============================] - 14s 141ms/step - loss: 3.8566 - binary_crossentropy: 0.6418 - jaccard_coef_int: 0.2786 - binary_accuracy: 0.8461 - val_loss: 8.8893 - val_binary_crossentropy: 1.2588 - val_jaccard_coef_int: 0.0014 - val_binary_accuracy: 0.7923
Epoch 4/50
100/100 [==============================] - 16s 159ms/step - loss: 3.5793 - binary_crossentropy: 0.6786 - jaccard_coef_int: 0.3120 - binary_accurac

Epoch 7/50
100/100 [==============================] - 11s 109ms/step - loss: 2.5810 - binary_crossentropy: 0.0718 - jaccard_coef_int: 0.1063 - binary_accuracy: 0.9891 - val_loss: 3.8989 - val_binary_crossentropy: 0.1585 - val_jaccard_coef_int: 0.0288 - val_binary_accuracy: 0.9780
Epoch 8/50
100/100 [==============================] - 11s 109ms/step - loss: 2.4880 - binary_crossentropy: 0.0802 - jaccard_coef_int: 0.1119 - binary_accuracy: 0.9894 - val_loss: 3.5798 - val_binary_crossentropy: 0.1689 - val_jaccard_coef_int: 0.0397 - val_binary_accuracy: 0.9790
Epoch 9/50
100/100 [==============================] - 11s 106ms/step - loss: 2.6830 - binary_crossentropy: 0.1094 - jaccard_coef_int: 0.0993 - binary_accuracy: 0.9865 - val_loss: 2.7842 - val_binary_crossentropy: 0.1543 - val_jaccard_coef_int: 0.0832 - val_binary_accuracy: 0.9823
Epoch 10/50
100/100 [==============================] - 11s 108ms/step - loss: 2.5311 - binary_crossentropy: 0.0984 - jaccard_coef_int: 0.1088 - binary_accura

Epoch 13/50
100/100 [==============================] - 11s 107ms/step - loss: 3.0544 - binary_crossentropy: 0.1411 - jaccard_coef_int: 0.0751 - binary_accuracy: 0.9799 - val_loss: 10.4579 - val_binary_crossentropy: 3.2872 - val_jaccard_coef_int: 0.0036 - val_binary_accuracy: 0.7002
Epoch 14/50
100/100 [==============================] - 11s 106ms/step - loss: 3.1226 - binary_crossentropy: 0.1474 - jaccard_coef_int: 0.0716 - binary_accuracy: 0.9784 - val_loss: 3.9813 - val_binary_crossentropy: 0.2217 - val_jaccard_coef_int: 0.0337 - val_binary_accuracy: 0.9653
Epoch 15/50
100/100 [==============================] - 11s 106ms/step - loss: 3.2692 - binary_crossentropy: 0.1481 - jaccard_coef_int: 0.0659 - binary_accuracy: 0.9766 - val_loss: 4.3947 - val_binary_crossentropy: 0.3950 - val_jaccard_coef_int: 0.0325 - val_binary_accuracy: 0.9421
Epoch 16/50
100/100 [==============================] - 10s 105ms/step - loss: 3.1378 - binary_crossentropy: 0.1521 - jaccard_coef_int: 0.0715 - binary_ac

...

Epoch 4/50
100/100 [==============================] - 13s 133ms/step - loss: 3.4225 - binary_crossentropy: 0.1573 - jaccard_coef_int: 0.0778 - binary_accuracy: 0.9585 - val_loss: 5.7426 - val_binary_crossentropy: 0.1223 - val_jaccard_coef_int: 0.0024 - val_binary_accuracy: 0.9813
Epoch 5/50
100/100 [==============================] - 13s 132ms/step - loss: 3.2958 - binary_crossentropy: 0.1705 - jaccard_coef_int: 0.0800 - binary_accuracy: 0.9612 - val_loss: 4.6227 - val_binary_crossentropy: 0.1312 - val_jaccard_coef_int: 0.0123 - val_binary_accuracy: 0.9814
Epoch 6/50
100/100 [==============================] - 13s 133ms/step - loss: 3.1779 - binary_crossentropy: 0.1831 - jaccard_coef_int: 0.0833 - binary_accuracy: 0.9639 - val_loss: 5.3860 - val_binary_crossentropy: 0.1345 - val_jaccard_coef_int: 0.0054 - val_binary_accuracy: 0.9813
Epoch 7/50
100/100 [==============================] - 13s 131ms/step - loss: 3.1346 - binary_crossentropy: 0.1966 - jaccard_coef_int: 0.0850 - binary_accurac

Epoch 10/50
100/100 [==============================] - 11s 114ms/step - loss: 3.5687 - binary_crossentropy: 0.5656 - jaccard_coef_int: 0.1053 - binary_accuracy: 0.9195 - val_loss: 4.1776 - val_binary_crossentropy: 0.5868 - val_jaccard_coef_int: 0.0553 - val_binary_accuracy: 0.9197
Epoch 11/50
100/100 [==============================] - 11s 112ms/step - loss: 3.5381 - binary_crossentropy: 0.5862 - jaccard_coef_int: 0.1100 - binary_accuracy: 0.9190 - val_loss: 4.3889 - val_binary_crossentropy: 0.5603 - val_jaccard_coef_int: 0.0381 - val_binary_accuracy: 0.9300
Epoch 12/50
100/100 [==============================] - 11s 110ms/step - loss: 3.3821 - binary_crossentropy: 0.5858 - jaccard_coef_int: 0.1218 - binary_accuracy: 0.9219 - val_loss: 4.0296 - val_binary_crossentropy: 0.5717 - val_jaccard_coef_int: 0.0601 - val_binary_accuracy: 0.9242
Epoch 13/50
100/100 [==============================] - 11s 112ms/step - loss: 3.3589 - binary_crossentropy: 0.5893 - jaccard_coef_int: 0.1217 - binary_acc

Epoch 16/50
100/100 [==============================] - 13s 134ms/step - loss: 3.4888 - binary_crossentropy: 0.6567 - jaccard_coef_int: 0.1075 - binary_accuracy: 0.9228 - val_loss: 3.7083 - val_binary_crossentropy: 0.6807 - val_jaccard_coef_int: 0.0931 - val_binary_accuracy: 0.9173
Epoch 17/50
100/100 [==============================] - 14s 136ms/step - loss: 3.4332 - binary_crossentropy: 0.6577 - jaccard_coef_int: 0.1103 - binary_accuracy: 0.9245 - val_loss: 4.0964 - val_binary_crossentropy: 1.0202 - val_jaccard_coef_int: 0.1075 - val_binary_accuracy: 0.8836
Epoch 18/50
100/100 [==============================] - 13s 135ms/step - loss: 3.4099 - binary_crossentropy: 0.6657 - jaccard_coef_int: 0.1126 - binary_accuracy: 0.9244 - val_loss: 3.8619 - val_binary_crossentropy: 0.6947 - val_jaccard_coef_int: 0.0789 - val_binary_accuracy: 0.9171
Epoch 19/50
100/100 [==============================] - 13s 134ms/step - loss: 3.3681 - binary_crossentropy: 0.6710 - jaccard_coef_int: 0.1163 - binary_acc

Epoch 22/50
100/100 [==============================] - 13s 133ms/step - loss: 2.3530 - binary_crossentropy: 0.0940 - jaccard_coef_int: 0.1685 - binary_accuracy: 0.9876 - val_loss: 5.1623 - val_binary_crossentropy: 0.0074 - val_jaccard_coef_int: 0.6315 - val_binary_accuracy: 0.9989
Epoch 23/50
100/100 [==============================] - 14s 136ms/step - loss: 2.6821 - binary_crossentropy: 0.0983 - jaccard_coef_int: 0.1475 - binary_accuracy: 0.9868 - val_loss: 5.7479 - val_binary_crossentropy: 0.0387 - val_jaccard_coef_int: 0.3031 - val_binary_accuracy: 0.9935
Epoch 24/50
100/100 [==============================] - 13s 133ms/step - loss: 2.5311 - binary_crossentropy: 0.0984 - jaccard_coef_int: 0.1448 - binary_accuracy: 0.9867 - val_loss: 7.7921 - val_binary_crossentropy: 0.0059 - val_jaccard_coef_int: 0.5750 - val_binary_accuracy: 0.9992
Epoch 25/50
100/100 [==============================] - 13s 134ms/step - loss: 2.5054 - binary_crossentropy: 0.0925 - jaccard_coef_int: 0.1671 - binary_acc

             precision    recall  f1-score   support

        0.0       0.99      1.00      0.99   1098608
        1.0       0.21      0.15      0.18      8592

avg / total       0.99      0.99      0.99   1107200





In [14]:
if __name__ == '__main__':
    results_dir = '/nfs/results/unet/dropouttest/80_80_16/results_prelim_unet_pl_fire'

    dict_img = {'width': 80, 'height': 80, 'stride': 80, 'padding': 16}
    dict_label = {'width': 80, 'height': 80, 'stride': 80}
    X_raw, Y_raw = geopatches.create_data(['/nfs/conditioned/planetlabs/1154314_2014-07-23_RE2_3A_Analytic_clipped.tif'],dict_img,\
                                        ['/nfs/conditioned/landfire/US_140FBFM13_3030_clipped.tif'],dict_label,\
                                         repeatlabel=6)
    # Planetlabs needs to be divided by max uint16 = 65535 to be between 0 and 1.0
    X_raw = X_raw / 65535

    #remove classes that are too small
    list_remove = []
    for x in np.asarray(np.unique(Y_raw, return_counts=True)).T:
        if x[1] < 400*36:
            list_remove.append(x[0])
    print('unique labels found:')
    list_unique = np.unique(Y_raw)
    print(list_unique)
    print('labels that do not meet threshold')
    print(list_remove)
    list_final = [x for x in list_unique if x not in list_remove]
    print('final labels')
    print(list_final)
    print('\n')
    
    # Variables
    batch_size = 128
    nb_epoch = 50
    steps_per_epoch = 100
    window_size = X_raw.shape[1]
    #nb_classes = 1
    nb_channels = X_raw.shape[3]
    cropping_len = 16
    
    # Build models
    for label in list_final:
        
        # Encode
        print('label:',label)
        X_train, X_val, X_test, Y_train, Y_val, Y_test = encode_binary_data({label: 1.0})
        print('X_train.shape','X_val.shape','X_test.shape','Y_train.shape','Y_val.shape','Y_test.shape',':')
        print(X_train.shape,X_val.shape,X_test.shape,Y_train.shape,Y_val.shape,Y_test.shape)
        
        # Model
        # https://www.pyimagesearch.com/2017/10/30/how-to-multi-gpu-training-with-keras-python-and-deep-learning/
        if len(list_gpus) <= 1:
            model = model_unet(window_size,window_size,nb_channels,cropping_len)
            model_gpu = model
        else:
            with tf.device("/cpu:0"):
                model = model_unet(window_size,window_size,nb_channels,cropping_len)
            model_gpu = multi_gpu_model(model, gpus=len(list_gpus))
        
        model_gpu.compile(optimizer=Nadam(lr=1e-3), loss=jaccard_coef_loss,\
                      metrics=['binary_crossentropy', jaccard_coef_int, "binary_accuracy"])
        
        history = History()
        ts_gen = generate_minibatch(X_train, Y_train, batch_size)
        #K.set_session(K.tf.Session(config=K.tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)))
        model_gpu.fit_generator(ts_gen,\
                    nb_epoch=nb_epoch,\
                    verbose=1,\
                    steps_per_epoch=steps_per_epoch,\
                    validation_data=(X_val, Y_val),\
                   callbacks=[history])

        # Calculate metrics off test
        Y_predict = model_gpu.predict(X_test)
        print(accuracy_score(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
        print(classification_report(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
        
        
        # Save
        suffix = str(label)
        save_model(model, "{batch}_{step}_{epoch}_{suffix}".format(batch=batch_size, step=steps_per_epoch, epoch=nb_epoch, suffix=suffix),results_dir)
        save_history(history, "{batch}_{step}_{epoch}_{suffix}".format(batch=batch_size, step=steps_per_epoch, epoch=nb_epoch, suffix=suffix),results_dir)
        
        # Calculate metrics off saved model
#         Y_predict = read_model("{batch}_{step}_{epoch}_{suffix}".format(batch=batch_size, step=steps_per_epoch, epoch=nb_epoch, suffix=suffix),results_dir).predict(X_test)
#         print(accuracy_score(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
#         print(classification_report(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
        
        print('\n')

unique labels found:
[ 1  2  4  5  7  8  9 91 93 98 99]
labels that do not meet threshold
[7, 93]
final labels
[1, 2, 4, 5, 8, 9, 91, 98, 99]


label: 1
{1: 1.0}
composition with label != 0
Y_train 0.8582930756843801 0.2713395732689211
composition with label != 0
Y_val 0.8428571428571429 0.2126607142857143
composition with label != 0
Y_test 0.861271676300578 0.2920628612716763
X_train.shape X_val.shape X_test.shape Y_train.shape Y_val.shape Y_test.shape :
(621, 112, 112, 5) (70, 112, 112, 5) (173, 112, 112, 5) (621, 80, 80, 1) (70, 80, 80, 1) (173, 80, 80, 1)


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:58: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:58: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<__main__...., callbacks=[<keras.ca..., validation_data=(array([[[..., epochs=50, steps_per_epoch=100, verbose=1)`


Epoch 1/50
100/100 [==============================] - 53s 528ms/step - loss: 6.3552 - binary_crossentropy: 0.6896 - jaccard_coef_int: 0.2381 - binary_accuracy: 0.7304 - val_loss: 5.6797 - val_binary_crossentropy: 0.9856 - val_jaccard_coef_int: 0.0945 - val_binary_accuracy: 0.7933
Epoch 2/50
100/100 [==============================] - 11s 113ms/step - loss: 4.6248 - binary_crossentropy: 0.8753 - jaccard_coef_int: 0.3831 - binary_accuracy: 0.7813 - val_loss: 8.0336 - val_binary_crossentropy: 1.1352 - val_jaccard_coef_int: 0.0053 - val_binary_accuracy: 0.7883
Epoch 3/50
100/100 [==============================] - 11s 112ms/step - loss: 4.4017 - binary_crossentropy: 0.9755 - jaccard_coef_int: 0.3944 - binary_accuracy: 0.7884 - val_loss: 7.9465 - val_binary_crossentropy: 2.3912 - val_jaccard_coef_int: 0.2833 - val_binary_accuracy: 0.5840
Epoch 4/50
100/100 [==============================] - 11s 111ms/step - loss: 4.2630 - binary_crossentropy: 1.0603 - jaccard_coef_int: 0.4011 - binary_accurac

Epoch 7/50
100/100 [==============================] - 14s 136ms/step - loss: 4.5781 - binary_crossentropy: 0.7643 - jaccard_coef_int: 0.1059 - binary_accuracy: 0.8590 - val_loss: 6.6568 - val_binary_crossentropy: 0.8787 - val_jaccard_coef_int: 0.0166 - val_binary_accuracy: 0.8361
Epoch 8/50
100/100 [==============================] - 14s 135ms/step - loss: 4.4811 - binary_crossentropy: 0.7845 - jaccard_coef_int: 0.1105 - binary_accuracy: 0.8621 - val_loss: 6.1007 - val_binary_crossentropy: 0.7374 - val_jaccard_coef_int: 0.0176 - val_binary_accuracy: 0.8636
Epoch 9/50
100/100 [==============================] - 14s 135ms/step - loss: 4.3111 - binary_crossentropy: 0.8232 - jaccard_coef_int: 0.1162 - binary_accuracy: 0.8681 - val_loss: 6.1095 - val_binary_crossentropy: 1.1964 - val_jaccard_coef_int: 0.0458 - val_binary_accuracy: 0.8115
Epoch 10/50
100/100 [==============================] - 13s 135ms/step - loss: 4.2825 - binary_crossentropy: 0.8391 - jaccard_coef_int: 0.1168 - binary_accura

Epoch 13/50
100/100 [==============================] - 14s 136ms/step - loss: 2.5590 - binary_crossentropy: 0.1411 - jaccard_coef_int: 0.1077 - binary_accuracy: 0.9836 - val_loss: 3.1395 - val_binary_crossentropy: 0.2065 - val_jaccard_coef_int: 0.0622 - val_binary_accuracy: 0.9777
Epoch 14/50
100/100 [==============================] - 13s 134ms/step - loss: 2.5918 - binary_crossentropy: 0.1523 - jaccard_coef_int: 0.1062 - binary_accuracy: 0.9828 - val_loss: 2.5493 - val_binary_crossentropy: 0.2368 - val_jaccard_coef_int: 0.1183 - val_binary_accuracy: 0.9746
Epoch 15/50
100/100 [==============================] - 14s 136ms/step - loss: 2.4774 - binary_crossentropy: 0.1469 - jaccard_coef_int: 0.1157 - binary_accuracy: 0.9839 - val_loss: 3.1248 - val_binary_crossentropy: 0.4775 - val_jaccard_coef_int: 0.0924 - val_binary_accuracy: 0.9541
Epoch 16/50
100/100 [==============================] - 13s 134ms/step - loss: 2.5310 - binary_crossentropy: 0.1573 - jaccard_coef_int: 0.1106 - binary_acc

...

Epoch 2/50
100/100 [==============================] - 13s 133ms/step - loss: 3.1069 - binary_crossentropy: 0.1343 - jaccard_coef_int: 0.1922 - binary_accuracy: 0.9596 - val_loss: 3.4891 - val_binary_crossentropy: 0.0786 - val_jaccard_coef_int: 0.0441 - val_binary_accuracy: 0.9818
Epoch 3/50
100/100 [==============================] - 13s 133ms/step - loss: 2.4466 - binary_crossentropy: 0.1557 - jaccard_coef_int: 0.1983 - binary_accuracy: 0.9616 - val_loss: 2.4881 - val_binary_crossentropy: 0.1420 - val_jaccard_coef_int: 0.1370 - val_binary_accuracy: 0.9714
Epoch 4/50
100/100 [==============================] - 13s 133ms/step - loss: 2.3463 - binary_crossentropy: 0.1739 - jaccard_coef_int: 0.2003 - binary_accuracy: 0.9623 - val_loss: 2.5512 - val_binary_crossentropy: 0.1366 - val_jaccard_coef_int: 0.1197 - val_binary_accuracy: 0.9756
Epoch 5/50
100/100 [==============================] - 13s 134ms/step - loss: 2.2384 - binary_crossentropy: 0.1855 - jaccard_coef_int: 0.2147 - binary_accurac

Epoch 8/50
100/100 [==============================] - 11s 115ms/step - loss: 2.8814 - binary_crossentropy: 0.9961 - jaccard_coef_int: 0.6148 - binary_accuracy: 0.8511 - val_loss: 3.1826 - val_binary_crossentropy: 1.1677 - val_jaccard_coef_int: 0.5746 - val_binary_accuracy: 0.8360
Epoch 9/50
100/100 [==============================] - 11s 115ms/step - loss: 2.8514 - binary_crossentropy: 1.0220 - jaccard_coef_int: 0.6178 - binary_accuracy: 0.8526 - val_loss: 3.2139 - val_binary_crossentropy: 1.2059 - val_jaccard_coef_int: 0.5747 - val_binary_accuracy: 0.8340
Epoch 10/50
100/100 [==============================] - 11s 114ms/step - loss: 2.8246 - binary_crossentropy: 1.0457 - jaccard_coef_int: 0.6237 - binary_accuracy: 0.8536 - val_loss: 2.9899 - val_binary_crossentropy: 1.1778 - val_jaccard_coef_int: 0.6046 - val_binary_accuracy: 0.8448
Epoch 11/50
100/100 [==============================] - 11s 113ms/step - loss: 2.8156 - binary_crossentropy: 1.0811 - jaccard_coef_int: 0.6291 - binary_accur

Epoch 14/50
100/100 [==============================] - 11s 114ms/step - loss: 0.6621 - binary_crossentropy: 0.0539 - jaccard_coef_int: 0.6464 - binary_accuracy: 0.9930 - val_loss: 1.0800 - val_binary_crossentropy: 0.1192 - val_jaccard_coef_int: 0.4274 - val_binary_accuracy: 0.9861
Epoch 15/50
100/100 [==============================] - 11s 115ms/step - loss: 0.6345 - binary_crossentropy: 0.0574 - jaccard_coef_int: 0.6485 - binary_accuracy: 0.9928 - val_loss: 1.0814 - val_binary_crossentropy: 0.1248 - val_jaccard_coef_int: 0.4268 - val_binary_accuracy: 0.9860
Epoch 16/50
100/100 [==============================] - 11s 114ms/step - loss: 0.5615 - binary_crossentropy: 0.0528 - jaccard_coef_int: 0.6672 - binary_accuracy: 0.9936 - val_loss: 1.0807 - val_binary_crossentropy: 0.1292 - val_jaccard_coef_int: 0.4271 - val_binary_accuracy: 0.9861
Epoch 17/50
100/100 [==============================] - 11s 113ms/step - loss: 0.5845 - binary_crossentropy: 0.0613 - jaccard_coef_int: 0.6576 - binary_acc

Epoch 20/50
100/100 [==============================] - 12s 122ms/step - loss: 2.9696 - binary_crossentropy: 0.1163 - jaccard_coef_int: 0.0972 - binary_accuracy: 0.9823 - val_loss: 5.3586 - val_binary_crossentropy: 0.2268 - val_jaccard_coef_int: 0.0063 - val_binary_accuracy: 0.9656
Epoch 21/50
100/100 [==============================] - 12s 123ms/step - loss: 2.7349 - binary_crossentropy: 0.1083 - jaccard_coef_int: 0.1151 - binary_accuracy: 0.9848 - val_loss: 9.1962 - val_binary_crossentropy: 0.1226 - val_jaccard_coef_int: 1.3545e-14 - val_binary_accuracy: 0.9861
Epoch 22/50
100/100 [==============================] - 12s 121ms/step - loss: 2.8083 - binary_crossentropy: 0.1140 - jaccard_coef_int: 0.1083 - binary_accuracy: 0.9840 - val_loss: 3.0642 - val_binary_crossentropy: 0.1641 - val_jaccard_coef_int: 0.0705 - val_binary_accuracy: 0.9748
Epoch 23/50
100/100 [==============================] - 12s 122ms/step - loss: 2.9418 - binary_crossentropy: 0.1643 - jaccard_coef_int: 0.1034 - binary

Epoch 49/50
100/100 [==============================] - 12s 123ms/step - loss: 3.1631 - binary_crossentropy: 0.1724 - jaccard_coef_int: 0.0899 - binary_accuracy: 0.9788 - val_loss: 5.2162 - val_binary_crossentropy: 0.4298 - val_jaccard_coef_int: 0.0181 - val_binary_accuracy: 0.9230
Epoch 50/50
100/100 [==============================] - 12s 122ms/step - loss: 2.6976 - binary_crossentropy: 0.1277 - jaccard_coef_int: 0.1203 - binary_accuracy: 0.9836 - val_loss: 9.4818 - val_binary_crossentropy: 0.1198 - val_jaccard_coef_int: 1.6524e-14 - val_binary_accuracy: 0.9877
0.9941248193641619
             precision    recall  f1-score   support

        0.0       0.99      1.00      1.00   1100808
        1.0       0.00      0.00      0.00      6392

avg / total       0.99      0.99      0.99   1107200





In [13]:
if __name__ == '__main__':
    results_dir = '/nfs/results/unet/dropouttest/80_80_16/results_prelim_unet_pl_landcover'

    dict_img = {'width': 80, 'height': 80, 'stride': 80, 'padding': 16}
    dict_label = {'width': 80, 'height': 80, 'stride': 80}
    X_raw, Y_raw = geopatches.create_data(['/nfs/conditioned/planetlabs/1154314_2014-07-23_RE2_3A_Analytic_clipped.tif'],dict_img,\
                                        ['/nfs/conditioned/landfire/escondido_landcover_clipped.tif'],dict_label,\
                                         repeatlabel=6)
    # Planetlabs needs to be divided by max uint16 = 65535 to be between 0 and 1.0
    X_raw = X_raw / 65535

    #remove classes that are too small
    list_remove = []
    for x in np.asarray(np.unique(Y_raw, return_counts=True)).T:
        if x[1] < 400*36:
            list_remove.append(x[0])
    print('unique labels found:')
    list_unique = np.unique(Y_raw)
    print(list_unique)
    print('labels that do not meet threshold')
    print(list_remove)
    list_final = [x for x in list_unique if x not in list_remove]
    print('final labels')
    print(list_final)
    print('\n')
    
    # Variables
    batch_size = 128
    nb_epoch = 50
    steps_per_epoch = 100
    window_size = X_raw.shape[1]
    #nb_classes = 1
    nb_channels = X_raw.shape[3]
    cropping_len = 16
    
    # Build models
    for label in list_final:
        
        # Encode
        print('label:',label)
        X_train, X_val, X_test, Y_train, Y_val, Y_test = encode_binary_data({label: 1.0})
        print('X_train.shape','X_val.shape','X_test.shape','Y_train.shape','Y_val.shape','Y_test.shape',':')
        print(X_train.shape,X_val.shape,X_test.shape,Y_train.shape,Y_val.shape,Y_test.shape)
        
        # Model
        # https://www.pyimagesearch.com/2017/10/30/how-to-multi-gpu-training-with-keras-python-and-deep-learning/
        if len(list_gpus) <= 1:
            model = model_unet(window_size,window_size,nb_channels,cropping_len)
            model_gpu = model
        else:
            with tf.device("/cpu:0"):
                model = model_unet(window_size,window_size,nb_channels,cropping_len)
            model_gpu = multi_gpu_model(model, gpus=len(list_gpus))
        
        model_gpu.compile(optimizer=Nadam(lr=1e-3), loss=jaccard_coef_loss,\
                      metrics=['binary_crossentropy', jaccard_coef_int, "binary_accuracy"])
        
        history = History()
        ts_gen = generate_minibatch(X_train, Y_train, batch_size)
        #K.set_session(K.tf.Session(config=K.tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)))
        model_gpu.fit_generator(ts_gen,\
                    nb_epoch=nb_epoch,\
                    verbose=1,\
                    steps_per_epoch=steps_per_epoch,\
                    validation_data=(X_val, Y_val),\
                   callbacks=[history])

        # Calculate metrics off test
        Y_predict = model_gpu.predict(X_test)
        print(accuracy_score(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
        print(classification_report(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
        
        
        # Save
        suffix = str(label)
        save_model(model, "{batch}_{step}_{epoch}_{suffix}".format(batch=batch_size, step=steps_per_epoch, epoch=nb_epoch, suffix=suffix),results_dir)
        save_history(history, "{batch}_{step}_{epoch}_{suffix}".format(batch=batch_size, step=steps_per_epoch, epoch=nb_epoch, suffix=suffix),results_dir)
        
        # Calculate metrics off saved model
#         Y_predict = read_model("{batch}_{step}_{epoch}_{suffix}".format(batch=batch_size, step=steps_per_epoch, epoch=nb_epoch, suffix=suffix),results_dir).predict(X_test)
#         print(accuracy_score(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
#         print(classification_report(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
        
        print('\n')

unique labels found:
[11 21 22 23 24 31 42 43 52 71 81 82 90 95]
labels that do not meet threshold
[42, 81, 95]
final labels
[11, 21, 22, 23, 24, 31, 43, 52, 71, 82, 90]


label: 11
{11: 1.0}
composition with label != 0
Y_train 0.06924315619967794 0.017379227053140097
composition with label != 0
Y_val 0.05714285714285714 0.019285714285714285
composition with label != 0
Y_test 0.046242774566473986 0.011065751445086706
X_train.shape X_val.shape X_test.shape Y_train.shape Y_val.shape Y_test.shape :
(621, 112, 112, 5) (70, 112, 112, 5) (173, 112, 112, 5) (621, 80, 80, 1) (70, 80, 80, 1) (173, 80, 80, 1)


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:58: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:58: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<__main__...., verbose=1, epochs=50, callbacks=[<keras.ca..., validation_data=(array([[[..., steps_per_epoch=100)`


Epoch 1/50
100/100 [==============================] - 21s 214ms/step - loss: 2.2361 - binary_crossentropy: 0.0657 - jaccard_coef_int: 0.6705 - binary_accuracy: 0.9873 - val_loss: 0.7261 - val_binary_crossentropy: 0.0578 - val_jaccard_coef_int: 0.6276 - val_binary_accuracy: 0.9902
Epoch 2/50
100/100 [==============================] - 13s 130ms/step - loss: 0.7692 - binary_crossentropy: 0.0247 - jaccard_coef_int: 0.7390 - binary_accuracy: 0.9953 - val_loss: 0.7879 - val_binary_crossentropy: 0.0576 - val_jaccard_coef_int: 0.5839 - val_binary_accuracy: 0.9895
Epoch 3/50
100/100 [==============================] - 13s 128ms/step - loss: 0.7173 - binary_crossentropy: 0.0416 - jaccard_coef_int: 0.7412 - binary_accuracy: 0.9913 - val_loss: 0.6363 - val_binary_crossentropy: 0.0706 - val_jaccard_coef_int: 0.6605 - val_binary_accuracy: 0.9911
Epoch 4/50
100/100 [==============================] - 13s 131ms/step - loss: 0.5053 - binary_crossentropy: 0.0260 - jaccard_coef_int: 0.7528 - binary_accurac

100/100 [==============================] - 13s 130ms/step - loss: 0.3935 - binary_crossentropy: 0.0480 - jaccard_coef_int: 0.7506 - binary_accuracy: 0.9954 - val_loss: 0.5644 - val_binary_crossentropy: 0.1186 - val_jaccard_coef_int: 0.6578 - val_binary_accuracy: 0.9910
Epoch 39/50
100/100 [==============================] - 13s 131ms/step - loss: 0.3467 - binary_crossentropy: 0.0475 - jaccard_coef_int: 0.7685 - binary_accuracy: 0.9954 - val_loss: 0.5563 - val_binary_crossentropy: 0.1156 - val_jaccard_coef_int: 0.6625 - val_binary_accuracy: 0.9911
Epoch 40/50
100/100 [==============================] - 13s 131ms/step - loss: 0.3579 - binary_crossentropy: 0.0521 - jaccard_coef_int: 0.7635 - binary_accuracy: 0.9951 - val_loss: 0.5600 - val_binary_crossentropy: 0.1183 - val_jaccard_coef_int: 0.6603 - val_binary_accuracy: 0.9910
Epoch 41/50
100/100 [==============================] - 13s 130ms/step - loss: 0.3934 - binary_crossentropy: 0.0517 - jaccard_coef_int: 0.7458 - binary_accuracy: 0.995

Epoch 15/50
100/100 [==============================] - 10s 103ms/step - loss: 4.4265 - binary_crossentropy: 1.4849 - jaccard_coef_int: 0.3105 - binary_accuracy: 0.7971 - val_loss: 4.8755 - val_binary_crossentropy: 1.5392 - val_jaccard_coef_int: 0.2049 - val_binary_accuracy: 0.7946
Epoch 16/50
100/100 [==============================] - 11s 105ms/step - loss: 4.3728 - binary_crossentropy: 1.5022 - jaccard_coef_int: 0.3138 - binary_accuracy: 0.7997 - val_loss: 4.7571 - val_binary_crossentropy: 1.5742 - val_jaccard_coef_int: 0.2262 - val_binary_accuracy: 0.7958
Epoch 17/50
 47/100 [=============>................] - ETA: 5s - loss: 4.3755 - binary_crossentropy: 1.5367 - jaccard_coef_int: 0.3197 - binary_accuracy: 0.7984

...

Epoch 24/50
100/100 [==============================] - 11s 107ms/step - loss: 2.3894 - binary_crossentropy: 0.5512 - jaccard_coef_int: 0.2747 - binary_accuracy: 0.9324 - val_loss: 3.2099 - val_binary_crossentropy: 0.8377 - val_jaccard_coef_int: 0.1918 - val_binary_accuracy: 0.9029
Epoch 25/50
100/100 [==============================] - 11s 106ms/step - loss: 2.3216 - binary_crossentropy: 0.5329 - jaccard_coef_int: 0.2803 - binary_accuracy: 0.9352 - val_loss: 3.7603 - val_binary_crossentropy: 0.8498 - val_jaccard_coef_int: 0.1090 - val_binary_accuracy: 0.9035
Epoch 26/50
100/100 [==============================] - 11s 107ms/step - loss: 2.2889 - binary_crossentropy: 0.5408 - jaccard_coef_int: 0.2880 - binary_accuracy: 0.9356 - val_loss: 3.6336 - val_binary_crossentropy: 0.9227 - val_jaccard_coef_int: 0.1480 - val_binary_accuracy: 0.8925
Epoch 27/50
100/100 [==============================] - 11s 107ms/step - loss: 2.2552 - binary_crossentropy: 0.5477 - jaccard_coef_int: 0.2977 - binary_acc

Epoch 1/50
100/100 [==============================] - 27s 272ms/step - loss: 5.8860 - binary_crossentropy: 0.1217 - jaccard_coef_int: 0.0439 - binary_accuracy: 0.9755 - val_loss: 30.3590 - val_binary_crossentropy: 0.0027 - val_jaccard_coef_int: 0.9875 - val_binary_accuracy: 1.0000
Epoch 2/50
100/100 [==============================] - 11s 105ms/step - loss: 4.0407 - binary_crossentropy: 0.0634 - jaccard_coef_int: 0.1127 - binary_accuracy: 0.9821 - val_loss: 30.0858 - val_binary_crossentropy: 0.0020 - val_jaccard_coef_int: 1.0000 - val_binary_accuracy: 1.0000
Epoch 3/50
100/100 [==============================] - 11s 107ms/step - loss: 3.4227 - binary_crossentropy: 0.0615 - jaccard_coef_int: 0.1570 - binary_accuracy: 0.9821 - val_loss: 28.1389 - val_binary_crossentropy: 2.9980e-04 - val_jaccard_coef_int: 1.0000 - val_binary_accuracy: 1.0000
Epoch 4/50
100/100 [==============================] - 11s 107ms/step - loss: 3.0929 - binary_crossentropy: 0.0564 - jaccard_coef_int: 0.1742 - binary_

Epoch 30/50
100/100 [==============================] - 11s 107ms/step - loss: 2.2158 - binary_crossentropy: 0.0878 - jaccard_coef_int: 0.2177 - binary_accuracy: 0.9884 - val_loss: 27.7991 - val_binary_crossentropy: 4.7903e-04 - val_jaccard_coef_int: 0.8750 - val_binary_accuracy: 1.0000
Epoch 31/50
100/100 [==============================] - 11s 106ms/step - loss: 2.0961 - binary_crossentropy: 0.0948 - jaccard_coef_int: 0.2149 - binary_accuracy: 0.9873 - val_loss: 26.9822 - val_binary_crossentropy: 2.0946e-04 - val_jaccard_coef_int: 1.0000 - val_binary_accuracy: 1.0000
Epoch 32/50
100/100 [==============================] - 11s 107ms/step - loss: 3.3666 - binary_crossentropy: 0.0894 - jaccard_coef_int: 0.1238 - binary_accuracy: 0.9820 - val_loss: 30.5204 - val_binary_crossentropy: 0.0087 - val_jaccard_coef_int: 0.2125 - val_binary_accuracy: 0.9978
Epoch 33/50
100/100 [==============================] - 11s 106ms/step - loss: 3.0966 - binary_crossentropy: 0.0810 - jaccard_coef_int: 0.1434 -

100/100 [==============================] - 11s 107ms/step - loss: 2.1764 - binary_crossentropy: 0.0351 - jaccard_coef_int: 0.1654 - binary_accuracy: 0.9931 - val_loss: 8.1248 - val_binary_crossentropy: 0.0188 - val_jaccard_coef_int: 0.0875 - val_binary_accuracy: 0.9977
Epoch 7/50
100/100 [==============================] - 11s 106ms/step - loss: 2.4213 - binary_crossentropy: 0.0518 - jaccard_coef_int: 0.1398 - binary_accuracy: 0.9903 - val_loss: 3.5168 - val_binary_crossentropy: 0.0189 - val_jaccard_coef_int: 0.1297 - val_binary_accuracy: 0.9976
Epoch 8/50
100/100 [==============================] - 11s 107ms/step - loss: 2.2459 - binary_crossentropy: 0.0662 - jaccard_coef_int: 0.1671 - binary_accuracy: 0.9876 - val_loss: 4.8321 - val_binary_crossentropy: 0.0973 - val_jaccard_coef_int: 0.0111 - val_binary_accuracy: 0.9793
Epoch 9/50
100/100 [==============================] - 11s 106ms/step - loss: 2.5344 - binary_crossentropy: 0.0558 - jaccard_coef_int: 0.1236 - binary_accuracy: 0.9875 -

100/100 [==============================] - 11s 106ms/step - loss: 1.3972 - binary_crossentropy: 0.0456 - jaccard_coef_int: 0.2955 - binary_accuracy: 0.9955 - val_loss: 11.3341 - val_binary_crossentropy: 0.0276 - val_jaccard_coef_int: 0.1000 - val_binary_accuracy: 0.9976
Epoch 36/50
100/100 [==============================] - 11s 108ms/step - loss: 1.4366 - binary_crossentropy: 0.0419 - jaccard_coef_int: 0.2828 - binary_accuracy: 0.9958 - val_loss: 5.7929 - val_binary_crossentropy: 0.0287 - val_jaccard_coef_int: 0.1028 - val_binary_accuracy: 0.9973
Epoch 37/50
100/100 [==============================] - 11s 107ms/step - loss: 1.3483 - binary_crossentropy: 0.0454 - jaccard_coef_int: 0.3041 - binary_accuracy: 0.9956 - val_loss: 3.5721 - val_binary_crossentropy: 0.0268 - val_jaccard_coef_int: 0.1286 - val_binary_accuracy: 0.9976
Epoch 38/50
100/100 [==============================] - 11s 106ms/step - loss: 1.3817 - binary_crossentropy: 0.0443 - jaccard_coef_int: 0.3021 - binary_accuracy: 0.99

In [14]:
if __name__ == '__main__':
    results_dir = '/nfs/results/unet/dropouttest/80_80_16/results_prelim_unet_pl_c24pl'

    dict_img = {'width': 80, 'height': 80, 'stride': 80, 'padding': 16}
    dict_label = {'width': 80, 'height': 80, 'stride': 80}
    X_raw, Y_raw = geopatches.create_data(['/nfs/conditioned/planetlabs/1154314_2014-07-23_RE2_3A_Analytic_clipped.tif'],dict_img,\
                                        ['/nfs/conditioned/planetlabs/ClusterRaster24_PlanetLabs.tif'],dict_label)
    # Planetlabs needs to be divided by max uint16 = 65535 to be between 0 and 1.0
    X_raw = X_raw / 65535

    #remove classes that are too small
    list_remove = []
#     for x in np.asarray(np.unique(Y_raw, return_counts=True)).T:
#         if x[1] < 400*36:
#             list_remove.append(x[0])
    print('unique labels found:')
    list_unique = np.unique(Y_raw)
    print(list_unique)
    print('labels that do not meet threshold')
    print(list_remove)
    list_final = [x for x in list_unique if x not in list_remove]
    print('final labels')
    print(list_final)
    print('\n')
    
    # Variables
    batch_size = 128
    nb_epoch = 50
    steps_per_epoch = 100
    window_size = X_raw.shape[1]
    #nb_classes = 1
    nb_channels = X_raw.shape[3]
    cropping_len = 16
    
    # Build models
    for label in list_final:
        
        # Encode
        print('label:',label)
        X_train, X_val, X_test, Y_train, Y_val, Y_test = encode_binary_data({label: 1.0})
        print('X_train.shape','X_val.shape','X_test.shape','Y_train.shape','Y_val.shape','Y_test.shape',':')
        print(X_train.shape,X_val.shape,X_test.shape,Y_train.shape,Y_val.shape,Y_test.shape)
        
        # Model
        # https://www.pyimagesearch.com/2017/10/30/how-to-multi-gpu-training-with-keras-python-and-deep-learning/
        if len(list_gpus) <= 1:
            model = model_unet(window_size,window_size,nb_channels,cropping_len)
            model_gpu = model
        else:
            with tf.device("/cpu:0"):
                model = model_unet(window_size,window_size,nb_channels,cropping_len)
            model_gpu = multi_gpu_model(model, gpus=len(list_gpus))
        
        model_gpu.compile(optimizer=Nadam(lr=1e-3), loss=jaccard_coef_loss,\
                      metrics=['binary_crossentropy', jaccard_coef_int, "binary_accuracy"])
        
        history = History()
        ts_gen = generate_minibatch(X_train, Y_train, batch_size)
        #K.set_session(K.tf.Session(config=K.tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)))
        model_gpu.fit_generator(ts_gen,\
                    nb_epoch=nb_epoch,\
                    verbose=1,\
                    steps_per_epoch=steps_per_epoch,\
                    validation_data=(X_val, Y_val),\
                   callbacks=[history])

        # Calculate metrics off test
        Y_predict = model_gpu.predict(X_test)
        print(accuracy_score(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
        print(classification_report(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
        
        
        # Save
        suffix = str(label)
        save_model(model, "{batch}_{step}_{epoch}_{suffix}".format(batch=batch_size, step=steps_per_epoch, epoch=nb_epoch, suffix=suffix),results_dir)
        save_history(history, "{batch}_{step}_{epoch}_{suffix}".format(batch=batch_size, step=steps_per_epoch, epoch=nb_epoch, suffix=suffix),results_dir)
        
        # Calculate metrics off saved model
#         Y_predict = read_model("{batch}_{step}_{epoch}_{suffix}".format(batch=batch_size, step=steps_per_epoch, epoch=nb_epoch, suffix=suffix),results_dir).predict(X_test)
#         print(accuracy_score(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
#         print(classification_report(np.rint(Y_test.flatten()), np.rint(Y_predict.flatten())))
        
        print('\n')

unique labels found:
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23]
labels that do not meet threshold
[]
final labels
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]


label: 0
{0: 1.0}
composition with label != 0
Y_train 0.9307568438003221 0.0945511272141707
composition with label != 0
Y_val 0.9571428571428572 0.08802008928571428
composition with label != 0
Y_test 0.9248554913294798 0.08844924132947977
X_train.shape X_val.shape X_test.shape Y_train.shape Y_val.shape Y_test.shape :
(621, 112, 112, 5) (70, 112, 112, 5) (173, 112, 112, 5) (621, 80, 80, 1) (70, 80, 80, 1) (173, 80, 80, 1)


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:58: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:58: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<__main__...., verbose=1, epochs=50, callbacks=[<keras.ca..., validation_data=(array([[[..., steps_per_epoch=100)`


Epoch 1/50
100/100 [==============================] - 34s 338ms/step - loss: 3.5322 - binary_crossentropy: 0.2279 - jaccard_coef_int: 0.3249 - binary_accuracy: 0.9058 - val_loss: 3.9595 - val_binary_crossentropy: 0.3765 - val_jaccard_coef_int: 0.0691 - val_binary_accuracy: 0.9134
Epoch 2/50
100/100 [==============================] - 13s 130ms/step - loss: 2.1348 - binary_crossentropy: 0.2089 - jaccard_coef_int: 0.4832 - binary_accuracy: 0.9312 - val_loss: 2.8092 - val_binary_crossentropy: 0.3976 - val_jaccard_coef_int: 0.2634 - val_binary_accuracy: 0.9087
Epoch 3/50
100/100 [==============================] - 13s 131ms/step - loss: 1.9272 - binary_crossentropy: 0.2225 - jaccard_coef_int: 0.5026 - binary_accuracy: 0.9341 - val_loss: 1.8801 - val_binary_crossentropy: 0.2451 - val_jaccard_coef_int: 0.4645 - val_binary_accuracy: 0.9340
Epoch 4/50
100/100 [==============================] - 13s 129ms/step - loss: 1.4389 - binary_crossentropy: 0.1726 - jaccard_coef_int: 0.5888 - binary_accurac

100/100 [==============================] - 13s 132ms/step - loss: 0.7126 - binary_crossentropy: 0.1878 - jaccard_coef_int: 0.7558 - binary_accuracy: 0.9736 - val_loss: 0.8187 - val_binary_crossentropy: 0.1859 - val_jaccard_coef_int: 0.6907 - val_binary_accuracy: 0.9720
Epoch 31/50
100/100 [==============================] - 13s 131ms/step - loss: 0.6639 - binary_crossentropy: 0.1712 - jaccard_coef_int: 0.7692 - binary_accuracy: 0.9756 - val_loss: 0.5760 - val_binary_crossentropy: 0.1246 - val_jaccard_coef_int: 0.7832 - val_binary_accuracy: 0.9798
Epoch 32/50
100/100 [==============================] - 13s 131ms/step - loss: 0.6696 - binary_crossentropy: 0.1755 - jaccard_coef_int: 0.7665 - binary_accuracy: 0.9754 - val_loss: 0.5108 - val_binary_crossentropy: 0.1089 - val_jaccard_coef_int: 0.8116 - val_binary_accuracy: 0.9819
Epoch 33/50
100/100 [==============================] - 13s 131ms/step - loss: 0.6430 - binary_crossentropy: 0.1664 - jaccard_coef_int: 0.7723 - binary_accuracy: 0.976

Epoch 7/50
100/100 [==============================] - 11s 109ms/step - loss: 1.8700 - binary_crossentropy: 0.2007 - jaccard_coef_int: 0.3621 - binary_accuracy: 0.9510 - val_loss: 1.7421 - val_binary_crossentropy: 0.1950 - val_jaccard_coef_int: 0.4283 - val_binary_accuracy: 0.9492
Epoch 8/50
100/100 [==============================] - 11s 108ms/step - loss: 1.7484 - binary_crossentropy: 0.1887 - jaccard_coef_int: 0.3911 - binary_accuracy: 0.9537 - val_loss: 1.6104 - val_binary_crossentropy: 0.1925 - val_jaccard_coef_int: 0.4595 - val_binary_accuracy: 0.9521
Epoch 9/50
100/100 [==============================] - 11s 107ms/step - loss: 1.6319 - binary_crossentropy: 0.1831 - jaccard_coef_int: 0.4191 - binary_accuracy: 0.9562 - val_loss: 1.6616 - val_binary_crossentropy: 0.2144 - val_jaccard_coef_int: 0.4415 - val_binary_accuracy: 0.9503
Epoch 10/50
100/100 [==============================] - 11s 109ms/step - loss: 1.4893 - binary_crossentropy: 0.1699 - jaccard_coef_int: 0.4549 - binary_accura

...

100/100 [==============================] - 11s 114ms/step - loss: 0.8399 - binary_crossentropy: 0.2202 - jaccard_coef_int: 0.7007 - binary_accuracy: 0.9704 - val_loss: 0.6582 - val_binary_crossentropy: 0.1520 - val_jaccard_coef_int: 0.7546 - val_binary_accuracy: 0.9772
Epoch 39/50
100/100 [==============================] - 11s 114ms/step - loss: 0.8336 - binary_crossentropy: 0.2203 - jaccard_coef_int: 0.7014 - binary_accuracy: 0.9707 - val_loss: 0.7189 - val_binary_crossentropy: 0.1842 - val_jaccard_coef_int: 0.7403 - val_binary_accuracy: 0.9746
Epoch 40/50
100/100 [==============================] - 11s 113ms/step - loss: 0.8189 - binary_crossentropy: 0.2160 - jaccard_coef_int: 0.7037 - binary_accuracy: 0.9714 - val_loss: 0.5926 - val_binary_crossentropy: 0.1250 - val_jaccard_coef_int: 0.7713 - val_binary_accuracy: 0.9800
Epoch 41/50
100/100 [==============================] - 11s 112ms/step - loss: 0.8012 - binary_crossentropy: 0.2115 - jaccard_coef_int: 0.7084 - binary_accuracy: 0.972

Epoch 15/50
100/100 [==============================] - 12s 122ms/step - loss: 0.8675 - binary_crossentropy: 0.1216 - jaccard_coef_int: 0.6061 - binary_accuracy: 0.9785 - val_loss: 0.8422 - val_binary_crossentropy: 0.0925 - val_jaccard_coef_int: 0.5839 - val_binary_accuracy: 0.9831
Epoch 16/50
100/100 [==============================] - 12s 123ms/step - loss: 0.8755 - binary_crossentropy: 0.1260 - jaccard_coef_int: 0.6037 - binary_accuracy: 0.9782 - val_loss: 1.0228 - val_binary_crossentropy: 0.1076 - val_jaccard_coef_int: 0.4901 - val_binary_accuracy: 0.9813
Epoch 17/50
100/100 [==============================] - 12s 121ms/step - loss: 0.8336 - binary_crossentropy: 0.1213 - jaccard_coef_int: 0.6191 - binary_accuracy: 0.9791 - val_loss: 0.7960 - val_binary_crossentropy: 0.0919 - val_jaccard_coef_int: 0.6005 - val_binary_accuracy: 0.9838
Epoch 18/50
100/100 [==============================] - 12s 123ms/step - loss: 0.8010 - binary_crossentropy: 0.1161 - jaccard_coef_int: 0.6301 - binary_acc

100/100 [==============================] - 12s 122ms/step - loss: 0.5033 - binary_crossentropy: 0.0922 - jaccard_coef_int: 0.7486 - binary_accuracy: 0.9871 - val_loss: 0.4115 - val_binary_crossentropy: 0.0539 - val_jaccard_coef_int: 0.7613 - val_binary_accuracy: 0.9919
Epoch 45/50
100/100 [==============================] - 12s 121ms/step - loss: 0.5009 - binary_crossentropy: 0.0929 - jaccard_coef_int: 0.7494 - binary_accuracy: 0.9871 - val_loss: 0.4599 - val_binary_crossentropy: 0.0686 - val_jaccard_coef_int: 0.7398 - val_binary_accuracy: 0.9905
Epoch 46/50
100/100 [==============================] - 12s 122ms/step - loss: 0.4848 - binary_crossentropy: 0.0897 - jaccard_coef_int: 0.7569 - binary_accuracy: 0.9875 - val_loss: 0.3861 - val_binary_crossentropy: 0.0478 - val_jaccard_coef_int: 0.7708 - val_binary_accuracy: 0.9925
Epoch 47/50
100/100 [==============================] - 12s 122ms/step - loss: 0.4757 - binary_crossentropy: 0.0887 - jaccard_coef_int: 0.7607 - binary_accuracy: 0.987

Epoch 21/50
100/100 [==============================] - 12s 123ms/step - loss: 1.0202 - binary_crossentropy: 0.2673 - jaccard_coef_int: 0.7012 - binary_accuracy: 0.9595 - val_loss: 1.3160 - val_binary_crossentropy: 0.4107 - val_jaccard_coef_int: 0.6547 - val_binary_accuracy: 0.9450
Epoch 22/50
100/100 [==============================] - 12s 124ms/step - loss: 1.0371 - binary_crossentropy: 0.2807 - jaccard_coef_int: 0.6959 - binary_accuracy: 0.9589 - val_loss: 1.0813 - val_binary_crossentropy: 0.3055 - val_jaccard_coef_int: 0.6970 - val_binary_accuracy: 0.9556
Epoch 23/50
100/100 [==============================] - 12s 122ms/step - loss: 0.9905 - binary_crossentropy: 0.2678 - jaccard_coef_int: 0.7087 - binary_accuracy: 0.9606 - val_loss: 0.9488 - val_binary_crossentropy: 0.2592 - val_jaccard_coef_int: 0.7331 - val_binary_accuracy: 0.9610
Epoch 24/50
100/100 [==============================] - 12s 124ms/step - loss: 0.9707 - binary_crossentropy: 0.2646 - jaccard_coef_int: 0.7146 - binary_acc

100/100 [==============================] - 12s 123ms/step - loss: 0.7100 - binary_crossentropy: 0.2384 - jaccard_coef_int: 0.7800 - binary_accuracy: 0.9716 - val_loss: 0.5518 - val_binary_crossentropy: 0.1820 - val_jaccard_coef_int: 0.8350 - val_binary_accuracy: 0.9772
0.9788899927745665
             precision    recall  f1-score   support

        0.0       0.99      0.99      0.99    980995
        1.0       0.91      0.91      0.91    126205

avg / total       0.98      0.98      0.98   1107200



